In [171]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler, Imputer, OneHotEncoder, LabelEncoder

%matplotlib inline


In [175]:
df = pd.read_csv('data/train.csv',sep=';')
df.head()



,VOIE_DEPOT,COUNTRY,SOURCE_BEGIN_MONTH,APP_NB,APP_NB_PAYS,APP_NB_TYPE,FISRT_APP_COUNTRY,FISRT_APP_TYPE,LANGUAGE_OF_FILLING,FIRST_CLASSE,...,oecd_NB_BACKWARD_NPL,IDX_ORIGIN,SOURCE_IDX_ORI,IDX_RADIC,SOURCE_IDX_RAD,VARIABLE_CIBLE,PRIORITY_MONTH,FILING_MONTH,PUBLICATION_MONTH,BEGIN_MONTH
0,EURO-PCT,US,D0,1,1,1,US,COMPAGNY,en,A61K9/48,...,0.000,1.000000,IMPUT,1.00,IMPUT,GRANTED,09/1995,08/1996,03/1997,09/1995
1,EURO-PCT,DE,D0,1,1,1,DE,COMPAGNY,de,C08G65/26,...,0.000,1.000000,IMPUT,1.00,IMPUT,GRANTED,11/1999,11/2000,05/2001,11/1999
2,VOIE DIRECTE,JP,D0,1,1,1,JP,COMPAGNY,en,G11B15/07,...,0.125,0.897778,CALC,1.00,IMPUT,GRANTED,06/1992,05/1993,12/1993,06/1992
3,VOIE DIRECTE,GB,D0,1,1,1,GB,COMPAGNY,en,F25B41/04,...,0.000,1.000000,IMPUT,1.00,IMPUT,GRANTED,05/1994,05/1995,11/1995,05/1994
4,VOIE DIRECTE,DE,D0,1,1,1,DE,COMPAGNY,de,H01F17/06,...,0.000,0.625000,CALC,0.25,CALC,GRANTED,12/1988,11/1989,06/1990,12/1988


In [176]:
df.dtypes


VOIE_DEPOT               object
COUNTRY                  object
SOURCE_BEGIN_MONTH       object
APP_NB                  float64
APP_NB_PAYS             float64
APP_NB_TYPE             float64
FISRT_APP_COUNTRY        object
FISRT_APP_TYPE           object
LANGUAGE_OF_FILLING      object
FIRST_CLASSE             object
NB_CLASSES                int64
NB_ROOT_CLASSES           int64
NB_SECTORS                int64
NB_FIELDS                 int64
TECHNOLOGIE_SECTOR       object
TECHNOLOGIE_FIELD        object
MAIN_IPC                 object
INV_NB                  float64
INV_NB_PAYS             float64
INV_NB_TYPE             float64
FISRT_INV_COUNTRY        object
FISRT_INV_TYPE           object
cited_n                 float64
cited_nmiss             float64
cited_age_min             int64
cited_age_median        float64
cited_age_max             int64
cited_age_mean          float64
cited_age_std           float64
SOURCE_CITED_AGE         object
NB_BACKWARD_NPL           int64
NB_BACKW

In [177]:
def unique(df):
    for col in df.columns:
        l = len(df[col].unique())
        
        print(col+' : ', l)
        print(df[col].unique())
        print('-----------------------------------------------------')

In [178]:
unique(df)

VOIE_DEPOT :  2
['EURO-PCT' 'VOIE DIRECTE']
-----------------------------------------------------
COUNTRY :  91
['US' 'DE' 'JP' 'GB' 'SE' nan 'IT' 'FR' 'EP' 'NO' 'NL' 'KR' 'CH' 'ES' 'WO'
 'YU' 'CN' 'DD' 'DK' 'AT' 'LU' 'FI' 'IS' 'CA' 'NZ' 'AU' 'IL' 'SG' 'AR' 'RU'
 'CZ' 'UA' 'IE' 'SU' 'BE' 'PL' 'IN' 'BR' 'HU' 'EG' 'SK' 'TR' 'RS' 'BG' 'AL'
 'SI' 'TW' 'ZA' 'LV' 'CU' 'CL' 'RO' 'GR' 'SM' 'EE' 'HK' 'PT' 'HR' 'MX' 'MY'
 'MT' 'IR' 'CO' 'CS' 'UY' 'EC' 'VE' 'SV' 'GE' 'BY' 'LB' 'KZ' 'MC' 'PH' 'TH'
 'BA' 'PK' 'ZW' 'MA' 'KP' 'MD' 'IB' 'EA' 'ML' 'ID' 'LT' 'HN' 'AE' 'TN' 'UZ'
 'JO']
-----------------------------------------------------
SOURCE_BEGIN_MONTH :  3
['D0' 'D1' nan]
-----------------------------------------------------
APP_NB :  16
[  1.   4.   2.   3.   7.  nan   5.   6.   9.  10.  13.   8.  17.  11.  16.
  14.]
-----------------------------------------------------
APP_NB_PAYS :  9
[  1.   2.   3.  nan   4.   5.   0.   7.  12.]
-----------------------------------------------------
APP_NB_TYP

# preprocessings

In [179]:
df = df.drop('cited_nmiss', axis=1) # because only 2 values : 0 and Nan 


def binarizer(x,string):
    if x == string:
        return 1
    else:
        return -1

df.VARIABLE_CIBLE = df.VARIABLE_CIBLE.apply(lambda row : binarizer(row,'GRANTED'))
df.SOURCE_IDX_RAD = df.SOURCE_IDX_RAD.apply(lambda row : binarizer(row,'CALC'))
df.SOURCE_IDX_ORI = df.SOURCE_IDX_ORI.apply(lambda row : binarizer(row,'CALC'))
df.SOURCE_CITED_AGE = df.SOURCE_CITED_AGE.apply(lambda row : binarizer(row,'CALC'))
df.VOIE_DEPOT = df.VOIE_DEPOT.apply(lambda row : binarizer(row,'VOIE DIRECTE'))

In [180]:
print(df.SOURCE_BEGIN_MONTH.unique())
df.SOURCE_BEGIN_MONTH = df.SOURCE_BEGIN_MONTH.str.extract('^D([0-1])').astype('float')
print(df.SOURCE_BEGIN_MONTH.unique())

['D0' 'D1' nan]
[  0.   1.  nan]


In [181]:
# extracting the month in date strings:

columns = ['PRIORITY_MONTH', 'FILING_MONTH', 'PUBLICATION_MONTH', 'BEGIN_MONTH']

for col in columns:
    df[col] = df[col].str.extract('^([0-9]{2})').astype('float')

In [182]:
print(len(df))
cols = df.columns

# selecting numerical and strings columns:
string_cols = []
categorical_num_cols = []
continuous_num_cols = []

for col in cols:
    if df[col].dtype == 'object':
        string_cols.append(col)
    elif df[col].dtype in ['float64', 'int64']:
        if len(df[col].unique()) > 40:
            continuous_num_cols.append(col)
        else:
            categorical_num_cols.append(col)
        

data_categorical_num = df[categorical_num_cols].values
data_continuous_num = df[continuous_num_cols].values
data_string = df[string_cols].values

print(categorical_num_cols)
print(continuous_num_cols)
print(string_cols)


print(data_categorical_num)
print(data_continuous_num)

259431
['VOIE_DEPOT', 'SOURCE_BEGIN_MONTH', 'APP_NB', 'APP_NB_PAYS', 'APP_NB_TYPE', 'NB_CLASSES', 'NB_ROOT_CLASSES', 'NB_SECTORS', 'NB_FIELDS', 'INV_NB', 'INV_NB_PAYS', 'INV_NB_TYPE', 'SOURCE_CITED_AGE', 'NB_BACKWARD_I', 'SOURCE_IDX_ORI', 'SOURCE_IDX_RAD', 'VARIABLE_CIBLE', 'PRIORITY_MONTH', 'FILING_MONTH', 'PUBLICATION_MONTH', 'BEGIN_MONTH']
['cited_n', 'cited_age_min', 'cited_age_median', 'cited_age_max', 'cited_age_mean', 'cited_age_std', 'NB_BACKWARD_NPL', 'NB_BACKWARD_XY', 'NB_BACKWARD_AUTRE', 'NB_BACKWARD_PL', 'NB_BACKWARD', 'pct_NB_IPC', 'pct_NB_IPC_LY', 'oecd_NB_ROOT_CLASSES', 'oecd_NB_BACKWARD_PL', 'oecd_NB_BACKWARD_NPL', 'IDX_ORIGIN', 'IDX_RADIC']
['COUNTRY', 'FISRT_APP_COUNTRY', 'FISRT_APP_TYPE', 'LANGUAGE_OF_FILLING', 'FIRST_CLASSE', 'TECHNOLOGIE_SECTOR', 'TECHNOLOGIE_FIELD', 'MAIN_IPC', 'FISRT_INV_COUNTRY', 'FISRT_INV_TYPE']
[[ -1.   0.   1. ...,   8.   3.   9.]
 [ -1.   0.   1. ...,  11.   5.  11.]
 [  1.   0.   1. ...,   5.  12.   6.]
 ..., 
 [  1.   0.   1. ...,  12.   

In [183]:
# filling Nan in continuous numerical columns :
imputer_continuous = Imputer(missing_values='NaN', strategy='mean', axis=0)

imputer_continuous.fit(data_continuous_num)
data_continuous_num = imputer_continuous.transform(data_continuous_num)

df1 = pd.DataFrame(data_continuous_num, columns=continuous_num_cols)

print(len(df1.dropna()))

259431


In [184]:
# filling Nan in categorical numerical columns :
imputer_categorical = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

imputer_categorical.fit(data_categorical_num)
data_categorical_num = imputer_categorical.transform(data_categorical_num)

df2 = pd.DataFrame(data_categorical_num, columns=categorical_num_cols)

print(len(df2.dropna()))

259431


In [185]:
# rebuilding dataFrame :
df3 = pd.DataFrame(data_string, columns=string_cols)
df = pd.concat([df1,df2,df3],axis=1)

In [186]:
unique(df)

cited_n :  59
[  2.02975834   6.           2.           4.           3.           1.           5.
  12.          14.           7.           8.           9.          11.          10.
  31.          23.          19.          25.          13.          64.          16.
  18.          65.          15.          24.          36.          38.          21.
  22.          29.          44.          35.          40.          32.          27.
  47.          61.          30.          59.          28.          48.          20.
  73.          50.          26.          49.          17.          37.          66.
  45.          39.          41.          52.          33.          89.          46.
  43.          34.          51.        ]
-----------------------------------------------------
cited_age_min :  6911
[ 7665.   256.   709. ...,  8792.  6984.  3803.]
-----------------------------------------------------
cited_age_median :  11544
[ 7665.   1147.   1541.5 ...,  5518.5  6984.   5160.5]
-------------

In [187]:
df = df.dropna()
print(len(df))

236273


In [188]:
df.head()

,cited_n,cited_age_min,cited_age_median,cited_age_max,cited_age_mean,cited_age_std,NB_BACKWARD_NPL,NB_BACKWARD_XY,NB_BACKWARD_AUTRE,NB_BACKWARD_PL,...,COUNTRY,FISRT_APP_COUNTRY,FISRT_APP_TYPE,LANGUAGE_OF_FILLING,FIRST_CLASSE,TECHNOLOGIE_SECTOR,TECHNOLOGIE_FIELD,MAIN_IPC,FISRT_INV_COUNTRY,FISRT_INV_TYPE
0,2.029758,7665,7665.0,7665,7665.000000,1035.157773,0,0,0,0,...,US,US,COMPAGNY,en,A61K9/48,III - Chemistry,16 - Pharmaceuticals,A61K,GB,(MISSING)
1,2.029758,7665,7665.0,7665,7665.000000,1035.157773,0,0,0,0,...,DE,DE,COMPAGNY,de,C08G65/26,III - Chemistry,"17 - Macromolecular chemistry, polymers",C08G,DE,(MISSING)
2,6.000000,256,1147.0,3365,1371.166667,1188.268558,1,3,8,11,...,JP,JP,COMPAGNY,en,G11B15/07,I - Electrical engineering,2 - Audio-visual technology,G11B,JP,COMPAGNY
3,2.029758,7665,7665.0,7665,7665.000000,1035.157773,0,3,0,3,...,GB,GB,COMPAGNY,en,F25B41/04,IV - Mechanical engineering,30 - Thermal processes and apparatus,F25B,GB,(MISSING)
4,2.000000,709,1541.5,2374,1541.500000,1177.332791,0,0,6,6,...,DE,DE,COMPAGNY,de,H01F17/06,I - Electrical engineering,"1 - Electrical machinery, apparatus, energy",H01F,DE,(MISSING)


In [189]:
# Encoding categorical features (in string type columns) :

LabEnc = {}
for col in string_cols:
    print(col)
    le = LabelEncoder()
    le.fit(df[col].values)
    df[col] = le.transform(df[col].values)
    
    LabEnc[col] = le


COUNTRY
FISRT_APP_COUNTRY
FISRT_APP_TYPE
LANGUAGE_OF_FILLING
FIRST_CLASSE
TECHNOLOGIE_SECTOR
TECHNOLOGIE_FIELD
MAIN_IPC
FISRT_INV_COUNTRY
FISRT_INV_TYPE


In [190]:
unique(df)

cited_n :  51
[  2.02975834   6.           2.           4.           3.           1.           5.
  14.           7.           8.           9.          11.          10.          31.
  23.          19.          25.          13.          64.          16.          12.
  65.          18.          15.          24.          36.          38.          21.
  22.          29.          44.          35.          40.          32.          30.
  59.          28.          48.          20.          26.          37.          17.
  66.          41.          27.          52.          33.          45.          61.
  43.          34.        ]
-----------------------------------------------------
cited_age_min :  6586
[ 7665.   256.   709. ...,  5619.  6984.  3803.]
-----------------------------------------------------
cited_age_median :  10953
[ 7665.   1147.   1541.5 ...,  4980.   6984.   5160.5]
-----------------------------------------------------
cited_age_max :  7950
[ 7665.  3365.  2374. ...,  6374. 

In [191]:
# dropping columns containing too many categories
df = df.drop(['COUNTRY', 'FISRT_APP_COUNTRY', 'LANGUAGE_OF_FILLING', 'FIRST_CLASSE', 'TECHNOLOGIE_FIELD', 'MAIN_IPC', 'FISRT_INV_COUNTRY'], axis=1)


In [ ]:
cat_cols = ['FISRT_APP_TYPE', 'TECHNOLOGIE_SECTOR', 'FISRT_INV_TYPE' ]

"""
OneHots = {}
for col in cat_cols:
    enc = OneHotEncoder()
    enc.fit(df[col].values)
    temp = enc_transform(df[col].values)
    
    df = df.drop(col)
    df
    
    OneHots[col] = enc
"""

In [146]:
df.head()

,cited_n,cited_age_min,cited_age_median,cited_age_max,cited_age_mean,cited_age_std,NB_BACKWARD_NPL,NB_BACKWARD_XY,NB_BACKWARD_AUTRE,NB_BACKWARD_PL,...,COUNTRY,FISRT_APP_COUNTRY,FISRT_APP_TYPE,LANGUAGE_OF_FILLING,FIRST_CLASSE,TECHNOLOGIE_SECTOR,TECHNOLOGIE_FIELD,MAIN_IPC,FISRT_INV_COUNTRY,FISRT_INV_TYPE
0,2.029758,7665,7665.0,7665,7665.000000,1035.157773,0,0,0,0,...,US,US,COMPAGNY,en,A61K9/48,III - Chemistry,16 - Pharmaceuticals,A61K,GB,(MISSING)
1,2.029758,7665,7665.0,7665,7665.000000,1035.157773,0,0,0,0,...,DE,DE,COMPAGNY,de,C08G65/26,III - Chemistry,"17 - Macromolecular chemistry, polymers",C08G,DE,(MISSING)
2,6.000000,256,1147.0,3365,1371.166667,1188.268558,1,3,8,11,...,JP,JP,COMPAGNY,en,G11B15/07,I - Electrical engineering,2 - Audio-visual technology,G11B,JP,COMPAGNY
3,2.029758,7665,7665.0,7665,7665.000000,1035.157773,0,3,0,3,...,GB,GB,COMPAGNY,en,F25B41/04,IV - Mechanical engineering,30 - Thermal processes and apparatus,F25B,GB,(MISSING)
4,2.000000,709,1541.5,2374,1541.500000,1177.332791,0,0,6,6,...,DE,DE,COMPAGNY,de,H01F17/06,I - Electrical engineering,"1 - Electrical machinery, apparatus, energy",H01F,DE,(MISSING)
